# DC MOTOR 2 CLICK 

**Setup code**

Imports the Mercury overlay from the Red Pitaya library and gives it an alias "FPGA". Creates an object called "overlay" which represents the Mercury overlay on the Red Pitaya board.

In [9]:
from redpitaya.overlay.mercury import mercury as FPGA
import time

# Initialize the FPGA overlay
overlay = FPGA()

# Get references to GPIO pins and LED
GPIO = FPGA.gpio
LED = FPGA.led

# Set the direction of mikrobus Pins to output
mikrobus1PwmPin = GPIO('p', 1, "out")
mikrobus1In1Pin = GPIO('n', 1, "out")
mikrobus1In2Pin = GPIO('n', 2, "out")
mikrobus1SlpPin = GPIO('p', 2, "out")

mikrobus2PwmPin = GPIO('p', 3, "out")
mikrobus2In1Pin = GPIO('n', 3, "out")
mikrobus2In2Pin = GPIO('n', 4, "out")
mikrobus2SlpPin = GPIO('p', 4, "out")

def pwm(pin, duty_cycle, num_seconds):
    period_us = 875
    pulse_us = (duty_cycle * period_us) / 100
    num_periods = 0
    
    while True:
        if num_periods ==  num_seconds *1000:
            break
        pin.write(True)
        time.sleep(pulse_us / 1000000)
        pin.write(False)
        time.sleep((period_us - pulse_us) / 1000000)
        num_periods += 1

def set_motor_mode(motor_mode):
    if motor_mode == "MODE_CCW":
        mikrobus1In1Pin.write(False)
        mikrobus1In2Pin.write(True)
        mikrobus1SlpPin.write(True)
    elif motor_mode == "MODE_CW":
        mikrobus1In1Pin.write(True)
        mikrobus1In2Pin.write(False)
        mikrobus1SlpPin.write(True)
    elif motor_mode == "MODE_STOP":
        mikrobus1In1Pin.write(False)
        mikrobus1In2Pin.write(False)
        mikrobus1SlpPin.write(True)
    elif motor_mode == "MODE_STANDBY":
        mikrobus1In1Pin.write(False)
        mikrobus1In2Pin.write(False)
        mikrobus1SlpPin.write(False)
    else:
        # Handle the default case
        pass

def pwm_sweep(pin, sweep_time, up_or_down):
    period_us = 875
    num_steps = int(sweep_time * 1000)  # Convert sweep time to milliseconds
    
    if up_or_down == "up":
        # The 0.15 is because the motor starts spinning at 15%
        for i in range(int(0.15 * num_steps), num_steps + 1):
            duty_cycle = (i * 100) / num_steps  # percent of power, zero to 100
            pulse_us = (duty_cycle * period_us) / 100
        
            # set pin state high
            pin.write(True)            
            # delay for pulse duration
            time.sleep(pulse_us / 1000000)

            # set pin state low
            pin.write(False)

            # delay for remaining period
            time.sleep((period_us - pulse_us) / 1000000)
    
    elif up_or_down == "down":
        # The 0.15 is because the motor starts spinning at 15%
        for i in range(num_steps, int(0.15 * num_steps) - 1, -1):
            duty_cycle = (i * 100) / num_steps  # percent of power, zero to 100
            pulse_us = (duty_cycle * period_us) / 100
        
            # set pin state high
            pin.write(True)
            
            # delay for pulse duration
            time.sleep(pulse_us / 1000000)

            # set pin state low
            pin.write(False)

            # delay for remaining period
            time.sleep((period_us - pulse_us) / 1000000)


Requested overlay is already loaded.
Overlay __del__ was activated.
But since there are garbage collection ordering issues, this code is commented out.


The example performs a PWM sweep to gradually increase and decrease the motor speed over a 10-second period. Afterward, it alternates the motor's direction and runs at 50% duty cycle for 3 seconds each in counter-clockwise, stopped, clockwise, and standby modes. The motor direction is set using an H-Bridge driver.

In [10]:
#sweep up or down to lower the current spike
set_motor_mode("MODE_CW")
pwm_sweep(mikrobus1PwmPin, 10, "up")

while True:
    # Set motor mode to counter-clockwise
    set_motor_mode("MODE_CCW")
    # Run motor at 50% duty cycle for 3 seconds
    pwm(mikrobus1PwmPin, 50, 3)

    # Set motor mode to stop
    set_motor_mode("MODE_STOP")
    # Run motor at 50% duty cycle for 3 seconds
    pwm(mikrobus1PwmPin, 50, 3)

    # Set motor mode to clockwise
    set_motor_mode("MODE_CW")
    # Run motor at 50% duty cycle for 3 seconds
    pwm(mikrobus1PwmPin, 50, 3)

    # Set motor mode to standby
    set_motor_mode("MODE_STANDBY")
    # Run motor at 50% duty cycle for 3 seconds
    pwm(mikrobus1PwmPin, 50, 3)

KeyboardInterrupt: 